In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes
!pip install gradio
!pip install -q -U git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-m8_bkgmq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-m8_bkgmq
  Resolved https://github.com/huggingface/transformers.git to commit 76a33a10923ccc1074917f6b6a1e719e626b7dc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8802691 sha256=23111ac072350206017c2b33a1b95c01dc57996facd240c275345edaf33cddff
  Stored in directory: /tmp/pip-ephem-wheel-cache-0v26wogq/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftConfig, PeftModel
import torch
from transformers import BitsAndBytesConfig

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_model_name = "TymofiiNas/results"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name, quantization_config=bnb_config, device_map={"": 0}
)

model = PeftModel.from_pretrained(model, adapter_model_name)
model.config.use_cache = True

tokenizer = AutoTokenizer.from_pretrained(base_model_name)


def generate_response(text):
    text = "<s> [INST]"+text+ "[/INST] " #note: for better results use this sentence before 'write a instagram post about'
    encoded_input = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encoded_input.to("cuda")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=400,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0][len(text):]


demo = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
)

demo.launch()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0af5a4f7ad7dcbcf27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
